In [2]:
import pandas as pd
import jsonlines

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns


In [3]:
# Zero-Shot Stance Detection: A Dataset and Model using Generalized Topic Representations

csv_file = "data/extra/vast_test.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(3)


,author,post,ori_topic,ori_id,new_topic,label,type_idx,new_id,arc_id,text,pos_text,text_s,topic,topic_str,seen?,contains_topic?,LexSim,Qte,Sarc,Imp,mlS,mlT
2010,casual observer,"Ginsburg's statement has no bearing on the separations of powers nor upon how she will consider cases under review, so I do not think what she said was improper. I do not think that Ginsberg did anything inappropriate since she did not in anyway refer to anything that will ever come before the Supreme Court. The Court has no ability to make any judgments that have any effect upon the election of Presidents nor of who is appointed to the Court. Such a personal opinion will never affect anythi...",elections,A3036-P,supreme court does not control election outcome,2,4,22529,19138566,"[[""ginsburg"", ""'s"", ""statement"", ""bearing"", ""separations"", ""powers"", ""upon"", ""consider"", ""cases"", ""review"", ""think"", ""said"", ""improper""], [""think"", ""ginsberg"", ""anything"", ""inappropriate"", ""since"", ""anyway"", ""refer"", ""anything"", ""ever"", ""come"", ""supreme"", ""court""], [""court"", ""ability"", ""make"", ""judgments"", ""effect"", ""upon"", ""election"", ""presidents"", ""appointed"", ""court""], [""personal"", ""opinion"", ""never"", ""affect"", ""anything"", ""addressed"", ""court""], [""anything"", ""stating"", ""something"", ""mass""...","[[""NNP"", ""POS"", ""NN"", ""NN"", ""NNS"", ""NNS"", ""IN"", ""VB"", ""NNS"", ""NN"", ""VB"", ""VBD"", ""JJ""], [""VB"", ""NNP"", ""NN"", ""JJ"", ""IN"", ""RB"", ""VB"", ""NN"", ""RB"", ""VB"", ""NNP"", ""NNP""], [""NNP"", ""NN"", ""VB"", ""NNS"", ""NN"", ""IN"", ""NN"", ""NNS"", ""VBN"", ""NNP""], [""JJ"", ""NN"", ""RB"", ""VB"", ""NN"", ""VBN"", ""NNP""], [""NN"", ""VBG"", ""NN"", ""JJ"", ""NN"", ""VBG"", ""VBZ"", ""VBN"", ""NN"", ""NNS"", ""JJ"", ""NN"", ""JJ"", ""NNS"", ""VBZ"", ""JJ"", ""NNP""]]",ginsburg 's statement bearing separations powers upon consider cases review think said improper. think ginsberg anything inappropriate since anyway refer anything ever come supreme court. court ability make judgments effect upon election presidents appointed court. personal opinion never affect anything addressed court. anything stating something mass media ignoring 's caught sport elections relevant analysis foreseeable consequences ends next president,"[""issuing"", ""id"", ""card""]",issuing id card,0,0,"[[""id"", ""card""]]",0,0,0,0,0
2026,Reuben Ryder,"Of course addicts should be committed for treatment. Unfortunately, we got in to this voluntaryism with mental health and addictive disorders, basically because it was cheaper. Everyone knows that it is, first: illegal to use band substances and second; that it is impossible to kick on your own, unless you are superman or woman. Let's give people a chance. Care about them. Do the right thing by them. Don't enable them.",treatment,A1990-P,treatment,2,4,22545,16635954,"[[""course"", ""addicts"", ""committed"", ""treatment""], [""unfortunately"", ""got"", ""voluntaryism"", ""mental"", ""health"", ""addictive"", ""disorders"", ""basically"", ""cheaper""], [""everyone"", ""knows"", ""first"", ""illegal"", ""use"", ""band"", ""substances"", ""second"", ""impossible"", ""kick"", ""unless"", ""superman"", ""woman""], [""let"", ""'s"", ""give"", ""people"", ""chance""], [""care""], [""right"", ""thing""], [""n't"", ""enable""]]","[[""RB"", ""NNS"", ""VBN"", ""NN""], [""RB"", ""VBD"", ""NN"", ""JJ"", ""NN"", ""JJ"", ""NNS"", ""RB"", ""JJR""], [""NN"", ""VBZ"", ""RB"", ""JJ"", ""VB"", ""NN"", ""NNS"", ""JJ"", ""JJ"", ""VB"", ""IN"", ""NN"", ""NN""], [""VB"", ""PRP"", ""VB"", ""NNS"", ""NN""], [""VB""], [""JJ"", ""NN""], [""RB"", ""VB""]]",course addicts committed treatment. unfortunately got voluntaryism mental health addictive disorders basically cheaper. everyone knows first illegal use band substances second impossible kick unless superman woman. let 's give people chance. care. right thing. n't enable,"[""gun""]",gun,0,0,[],0,0,0,0,0
1786,lofie,"I me

In [5]:
# Detecting Polarized Topics Using Partisanship-aware Contextualized Topic Embeddings

target_sources = ['CNN', 'Fox News', 'Huffington Post US', 
                  'Breitbart', 'The New York Times', 'New York Post']

aylien_json_path = 'data/extra/aylien_covid_news_data.jsonl'
articles = []
i = 0
with jsonlines.open(aylien_json_path) as reader:
    for obj in reader:
        source =  obj['source']['name']
        if source in target_sources:
            summary = obj['summary']['sentences']
            country = 'US'
            info = (obj['title'], source, obj['published_at'], obj['language'], country, tuple(summary))
            articles.append(info)
            i += 1
            if i % 2000 == 0:
                print(i, source)
data_raw = pd.DataFrame(articles, columns=['title', 'source', 'date', 'language', 'country', 'summary'])

2000 CNN
4000 CNN
6000 New York Post
8000 CNN
10000 The New York Times
12000 Breitbart
14000 The New York Times
16000 CNN
18000 Fox News
20000 CNN
22000 CNN
24000 The New York Times
26000 Fox News
28000 CNN
30000 Huffington Post US
32000 CNN
34000 CNN
36000 Fox News
38000 Fox News
40000 Fox News
42000 New York Post
44000 CNN
46000 Huffington Post US
48000 CNN
50000 Fox News
52000 CNN
54000 New York Post
56000 New York Post
58000 CNN
60000 New York Post
62000 Huffington Post US
64000 CNN
66000 The New York Times
68000 Fox News
70000 CNN
72000 Fox News
74000 New York Post


In [6]:
import copy
data = copy.deepcopy(data_raw)
data.sample(5)

,title,source,date,language,country,summary
5965,"If you can't wait for Amazon shipping, shop these other online retailers",CNN,2020-03-27 19:19:46+00:00,en,US,"(But if you have a favorite clothing, game, or homegoods shop nearby, this is a great time to check if it's completing online orders; many small stores are waiving or reducing shipping fees in the wake of social distancing, and you'll be supporting your local economy., HM\nHM has a very chic, affordable home section, and is offering free shipping and returns on all orders., Sporting goods\n \nREI\nREI is offering free standard shipping on all online orders, even as its physical stores remain..."
27330,"Georgia man charged with fraud for falsely claiming he was infected with Covid-19, federal prosecutors say",CNN,2020-05-23 02:09:59+00:00,en,US,"(That cost the corporation more than $100,000 and forced the unnecessary quarantine of several other employees, according to the US attorney's office., An affidavit filed by an FBI agent said the company repeatedly contacted Davis in an effort to get a copy of a positive Covid-19 test but he never provided it., Davis has since admitted he did not have Covid-19, according to the US attorney's office., The case is part of Georgia's Coronavirus (Covid-19) Fraud Task Force, which seeks to help p..."
9840,Lost Sense of Smell May Be Peculiar Clue to Coronavirus Infection,The New York Times,2020-03-23 00:14:09+00:00,en,US,"(Hendrik Streeck, a German virologist from the University of Bonn who went from house to house in the country’s Heinsberg district to interview coronavirus patients, has said in interviews that at least two-thirds of the more than 100 he talked to with mild disease reported experiencing loss of smell and taste lasting several days., Another physician who studied a cluster of coronavirus patients in Germany said in an email that roughly half of the patients had experienced a smell or taste di..."
67648,Georgia Bill Would Stop Officials From Mailing Absentee Ballot Applications,Huffington Post US,2020-06-24 21:00:52+00:00,en,US,"(ATLANTA (AP) — Republicans controlling a Georgia House committee approved legislation Wednesday that would prevent election officials from proactively sending mail ballot request forms to voters ahead of an election., To protect voting rights during the coronavirus pandemic, Georgia Secretary of State Brad Raffensperger, a Republican, sent absentee ballot applications to nearly 7 million active registered voters for the state’s June 9 primary elections, enabling huge numbers to avoid having..."
23026,"China orders 1,4000 military doctors, nurses to treat coronavirus",New York Post,2020-02-02 17:52:54+00:00,en,US,"(China will deploy 1,400 medical personnel from the military to treat patients infected with the deadly coronavirus, according to reports., President Xi Jinping ordered the military Sunday to begin airlifting hundreds of doctors and nurses, as well as 58 tons of medical supplies to the epicenter of the deadly virus, the Bangkok Post reported., The medics will handle patients at the recently-completed Huoshenshan Hospital, which was built in a matter of days to specialize in the coronavirus, ..."


In [7]:
# Analyzing Political Bias and Unfairness in News Articles at Different Levels of Granularity
# Political Ideology and Polarization: A Multi-dimensional Approach

import json

with open("data/extra/released_data.json", "r") as file:
    j_string = file.readline()
    jsons = []
    while j_string:
        data = json.loads(j_string)
        jsons.append(data)
        j_string = file.readline()
        
df = pd.DataFrame(jsons)

df.sample()

,source,title,event_id,adfontes_fair,adfontes_political,allsides_bias,content,misc
4119,CNN (Web News),Bernie Sanders: Hillary Clinton is not 'qualified' to be president,0,bias,neutral,From the Left,"Watch CNN and NY1's Democratic debate, moderated by Wolf Blitzer, Thursday, April 14 at 9 p.m. ET.\nWashington (CNN)Bernie Sanders is standing by his criticism that Hillary Clinton isn't ""qualified"" to be president as the Democratic presidential candidates engage in an increasingly heated battle ahead of the New York primary.\nBut by Thursday night, the Vermont senator said he didn't want to engage in a tit-for-tat with Clinton, telling CBS News they should be ""debating the issues facing the...","{'time': '2016-04-08 00:00:00', 'topics': 'Election: Presidential', 'author': ['Theodore Schleifer']}"


In [8]:
# Detecting White Supremacist Hate Speech Using Domain Specific Word Embedding With Deep Learning and BERT (48 citations)
csv_file = "data/extra/WhiteSupremacistDataset.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(2)

,input.text,annot1,annot2,annot3,Coded Labels,Coded Labels.1,Coded Labels.2,Binary Coded Labels,Binary Coded Labels.1,Binary Coded Labels.2,Voting and Final Labels
512,Epic troll bro,Other hate Speech,Neutral,Explicit White Supremacism,3,0,2,1,0,1,1
790,The West is importing the people with such cultural background by hundreds of thousands Can anyone tell me Why On whose orders For what purpose How come we the people have no say in such fundamental changes of our civilization Would anyone bother to address any of these,Implicit White Supremacism,Neutral,Neutral,1,0,0,1,0,0,0


In [9]:
# Automated Hate Speech Detection and the Problem of Offensive Language (2201 citations)

csv_file = "data/extra/labeled_data.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
8515,8751,6,1,1,4,2,Conan Obrien needs to get a life with that $80 million..monkey food maybe.
16516,16895,3,0,3,0,1,"RT @MrBrenBren: ""@1waynostra: These bitches crazzzyyyyyy"" maaannnnn"
4033,4153,3,0,3,0,1,@Marilyn_Munster Ha!!!! The shyt niggas tell hoes 2 ho wanna be more...
23437,23926,3,0,1,2,2,You wanna find hoes on here? Just follow the chicks who reply with heart eyes under sex gifs
12349,12661,3,0,3,0,1,"Let a bitch try and set me up , she gone be stuck between a rock nd dis mufuckin Glock! MY BITCH BAD SHE TERRIBLE &#128044;"


In [10]:
# Political Polarization During the COVID-19 Pandemic (51 citations)
csv_file = "data/extra/corona_comp_1720.dta"
df = pd.read_stata(csv_file)

# Display the DataFrame
df.sample(5)

,Vignette,edu2,sex,age,income,cducsu,spd,fdp,green,left,afd,other,dnv,feel,group,vote,time
3742,1.0,low,female,51.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0,AfD,1,2017
3739,11.0,low,female,19.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,49.0,SPD,5,2017
3377,10.0,low,female,24.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Left Party,8,2017
1769,15.0,low,male,44.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TUR,8,2020-W2
1358,14.0,low,male,68.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,Nationalist,1,2020-W1


In [11]:
# Knowledge Enhanced Masked Language Model for Stance Detection

csv_file = "data/extra/2020_twitter/biden_stance_test_public.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,tweet_id,text,label
235,1295755556553195520,a couple photos from yesterday's pro-life caucus with @USER. we are proud to partner with democrats for life in order to confront abortion extremism within the dnc! #weshowup #demconvention HTTP,NONE
242,1293344396096811009,@USER @USER haha - she is already breaking your hand. what will she break on you next?\n#trump2020 \n#dementiajoe HTTP,AGAINST
72,1295520389368614914,@USER socialism is what the democratic party is trying to bring down on our own country!!! drain that swamp!! #trump2020 #trumpsaveamerica,AGAINST
315,1293446102784647168,win what? the competition for worst and most awkward handshake ever? @USER @USER \n\n#bidenharris2020 #biden2020 #kamalaharris #2020election #handshake \n\n🖖🏻 HTTP,NONE
158,1311068466942750721,@USER @USER and @USER have to sow doubt and cheat and will say after the debate that biden cheated - as they already are prior to. his cult will believe whatever he says. it’s so disturbing,FAVOR


In [12]:
# Knowledge Enhanced Masked Language Model for Stance Detection

csv_file = "data/extra/2020_twitter/trump_stance_test_public.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,tweet_id,text,label
153,1311100627913760768,@USER so am i!!! this republican is ridin’ with biden!,AGAINST
330,1311179103341481984,@USER how could you hear anything over trump's loud and constant interruptions?,AGAINST
296,1303725758750302210,"#donaldtrump admitted he knew weeks before the first confirmed us coronavirus death that the virus was dangerous, airborne, highly contagious and ""more deadly than even your strenuous flus,"" and that he repeatedly played it down publicly. HTTP",AGAINST
159,1258818913653014528,HTTP\n\n#voteblue #voteblue2020 #votebluenomatterwho #votebluelikeyourlifedepends \n#votebluetoendthisnightmare \n#votebluetosaveamerica \n#votebluetosavetheplanet \n#votethemout \n#votelikeablackwoman come on lets go! #vote @USER,AGAINST
143,1298095467327188993,@USER look at at little jimmy squirming because he knows he's going to get four more years of trump #trump2020 #snowflake #maga2020 #trump12moreyears,FAVOR
